In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Activation
import numpy as np
import pretty_midi
import os
import matplotlib.pyplot as plt

In [2]:
np.random.seed(42)
tf.random.set_seed(42)

In [3]:
# CHANGE PATH AS NECESSARY
PATH = 'music_data'

In [8]:
def load_midi_files(path):
    songs = []

    with tf.device('/GPU:0'):
        for folder in os.listdir(path):
            if folder == '..LICENSING':
                continue
            if not os.path.isdir(os.path.join(path, folder)):
                continue
            for file_name in os.listdir(os.path.join(path, folder)):
                try:
                    midi = pretty_midi.PrettyMIDI(os.path.join(path, folder, file_name))
                    songs.append(midi)
                except Exception as e:
                    continue
    return songs

In [9]:
songs = load_midi_files(PATH)

/Users/nathanvandrie/Documents/VSCode-Workspace/GenAI/harmony-gen/.venv/lib/python3.12/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [10]:
from collections import defaultdict

def get_codes():
    program_count = defaultdict(int)
    for song in songs:
        unique_programs = {instrument.program for instrument in song.instruments}
        for program in unique_programs:
            program_count[program] += 1

    return dict(program_count)

codes = get_codes()
print(sorted(codes, key=codes.get, reverse=True))

[0, 33, 25, 48, 27, 29, 35, 30, 52, 32, 24, 26, 28, 49, 1, 50, 53, 65, 61, 73, 18, 4, 5, 16, 56, 54, 81, 66, 11, 62, 34, 57, 119, 89, 17, 38, 39, 60, 90, 75, 88, 2, 87, 40, 80, 82, 22, 71, 127, 8, 68, 63, 7, 3, 51, 46, 100, 31, 95, 6, 67, 91, 36, 85, 47, 45, 37, 99, 94, 9, 64, 84, 78, 12, 118, 44, 21, 58, 120, 72, 122, 10, 14, 42, 55, 79, 59, 93, 43, 74, 126, 117, 104, 19, 103, 102, 98, 105, 41, 92, 70, 116, 83, 23, 69, 20, 114, 96, 115, 124, 77, 125, 76, 97, 101, 108, 110, 13, 106, 112, 107, 15, 109, 86, 113, 123, 121, 111]


In [ ]:
# import pickle

# def save_data(data):
#     with open('music_data.pkl', 'wb') as f:
#         pickle.dump(data, f)

# def load_data():
#     with open('music_data.pkl', 'rb') as f:
#         loaded_data = pickle.load(f)
#     return loaded_data

In [15]:
# Program codes for midi instruments. Add more as necessary for filtering.
program_codes = {
    'VOICE_LEAD': 85,
}

def filter_data(data):
    melodies = []
    accompaniments = []
    for midi in data:
        melody = None
        accompaniment = []
        for i in midi.instruments:
            if i.program == program_codes['VOICE_LEAD'] and not melody:
                melody = i.notes
            else:
                accompaniment.append(i.notes)
        if melody:
            melodies.append(melody)
            accompaniments.append(accompaniment)
    return melodies, accompaniments

In [16]:
melodies, accompaniments = filter_data(songs)

In [14]:
# Convert notes to numeric sequences (pitch and timing encoding)
def notes_to_sequences(notes, seq_length=50):
    sequences = []
    for i in range(0, len(notes) - seq_length):
        seq = [(note.pitch, note.start, note.end) for note in notes[i:i+seq_length]]
        sequences.append(seq)
    return sequences

In [39]:
# Process and convert notes to sequences
seq_length = 50
melody_sequences = [notes_to_sequences(m, seq_length) for m in melodies]
accompaniment_sequences = [[notes_to_sequences(notes, seq_length) for notes in a] for a in accompaniments]
print(len(melody_sequences))
print(len(accompaniment_sequences))

In [41]:
# Flatten nested lists and pad sequences
melody_sequences = np.array([seq for sublist in melody_sequences for seq in sublist])
accompaniment_sequences = np.array([seq for a in accompaniment_sequences for sublist in a for seq in sublist])

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (488,) + inhomogeneous part.

In [35]:
# Normalize pitches for better training stability
def normalize_sequences(sequences):
    sequences = np.array(sequences)
    pitch_data = sequences[..., 0]
    timing_data = sequences[..., 1:]
    pitch_data = pitch_data / 127.0  # Normalize pitch (MIDI range is 0-127)
    return np.concatenate([pitch_data[..., None], timing_data], axis=-1)

In [36]:
print(f"Melody sequences shape: {melody_sequences.shape}")
melody_sequences = normalize_sequences(melody_sequences)
print(f"Accompaniment sequences shape: {accompaniment_sequences.shape}")
accompaniment_sequences = normalize_sequences(accompaniment_sequences)

Melody sequences shape: (13171200,)
Accompaniment sequences shape: (117085550, 3)
